In [37]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
from langdetect import detect, DetectorFactory

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIIT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [38]:
filename = 'amazon_in_urls.txt'
file = open('amazon_in_urls.txt')
LIST=re.split("\r\n|\n",file.read())
file.close()
# for i in LIST:
#     print(i.split('~')[-1])
#     url.append(i.split('~')[-1])
base_url = LIST[0].split('~')[-1]
product_name = str(LIST[1])
filename.split(".")

['amazon_in_urls', 'txt']

In [39]:
reviewlist = []
domains = [".com", ".co.uk", ".ca", ".in", ".co.mx", ".com.au", ".co.jp", ".sg", ".nl", ".de", ".fr", ".it", ".es", ".com.br"]
# domains = [".co.jp"]

In [40]:
def get_soup(url):
    r = requests.get('http://localhost:8050/render.html', params={'url': url, 'wait': 2})
    soup = BeautifulSoup(r.text, 'html.parser')
    return soup

In [41]:
def get_reviews(soup):
    reviews = soup.find_all('div', {'data-hook': 'review'})
    try:
        for item in reviews:
            product_title = soup.title.text.strip()
            product_title = product_title.split("Intel", 1)[-1].strip()
            product_title = "Intel " + product_title

            review_date = item.find('span', {'data-hook': 'review-date'}).text.strip()
            review_date = ' '.join(review_date.split()[-3:])

            rating_text = item.find('i', {'data-hook': 'review-star-rating'}).text.strip()
            rating_value = re.search(r'(\d+(\.\d+)?)', rating_text).group(1)
            rating = float(rating_value)

            review = {
                'Product': product_title,
                'Rating': rating,
                'Date': review_date,
                'Review': item.find('span', {'data-hook': 'review-body'}).text.strip(),
            }
            reviewlist.append(review)
    except Exception as e:
        print(f"An error occurred: {e}")

In [42]:
for y in domains:
    for x in range(1,11):
        url = re.sub(r'www\.amazon\.[a-z\.]+', f'www.amazon{y}', base_url)
        url = f'{url}&pageNumber={x}'
        soup = get_soup(url)
        print(f'Getting page: {x} from {y}')
        initial_length = len(reviewlist)
        get_reviews(soup)
        print(len(reviewlist))
        print(url)
        if len(reviewlist) == initial_length:
            break
        if not soup.find('li', {'class': 'a-disabled a-last'}):
            pass
        else:
            break

Getting page: 1 from .com
10
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=1
Getting page: 2 from .com
20
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=2
Getting page: 3 from .com
30
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=3
Getting page: 4 from .com
40
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=4
Getting page: 5 from .com
50
https://www.amazon.com/Intel-i9-10900K-Desktop-Processor-Unlocked/product-reviews/B086MHSTVD/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber=5
Getting page: 6 from

In [43]:
DetectorFactory.seed = 0

custom_words = [
    "against", "above", "below", "out", "under", "again",
    "no", "nor", "not", "don't", "should", "didn't",
    "does", "doesn't", "hadn't", "hasn't", "haven't",
    "isn't", "mustn", "mustn't", "should", "shouldn't",
    "was", "wasn't", "weren", "weren't", "won't", "wouldn't",
    "very", "couldn't"
]

stop_words = set(stopwords.words('english'))
for word in custom_words:
    if word in stop_words:
        stop_words.remove(word)

lemmatizer = WordNetLemmatizer()

for i in range(len(reviewlist)):
    review_dict = reviewlist[i]
    review = review_dict['Review']
    
    # Detect the language of the review
    try:
        language = detect(review)
        if language == 'ja':  # Skip Japanese reviews
            print(f"Skipping Japanese review: {review}")
            continue
    except Exception as e:
        print(f"Error detecting language: {e}")
        continue
    
    # Remove non-alphabetic characters
    review = re.sub(r'[^A-Za-z\s]', '', review)
    
    # Convert to lowercase
    review = review.lower()
    
    # Tokenize and remove stopwords
    words = word_tokenize(review)
    filtered_words = [word for word in words if word not in stop_words]
    
    # Lemmatize
    lemmatized_review = ' '.join([lemmatizer.lemmatize(word) for word in filtered_words])
    
    review_dict['Review'] = lemmatized_review


Skipping Japanese review: 現在Ryzenが品薄＋値上がり傾向にあるので、こちらを購入してみました。ネットで性能が悪いと酷評を付けられているので不安でしたが、どんなゲームをしていてもCPU使用率が30%を超えることはまずありません。FPSも60どころか100を割ることは、今のところありません。QHD環境でRTX3080と合わせて使用しています。どのCPUも正直過剰スペックであり、別にRyzenにこだわる必要はなくどちらでも良いんじゃないかと思いました。
Skipping Japanese review: よかたよねぇ
Skipping Japanese review: 無事に到着しました
Skipping Japanese review: i7-8700から乗り換えました。ベンチマークを測ってみると2倍くらいの性能が出ていました。私はほとんどゲームしかしないのですが、いくつかゲームを起動しても前者では時々CPU使用率が100％になり重くなることがあったのですが後者に変えてからは、そのようなことも無くて満足です。ですが、まだ買ってから１ヶ月ほどしか経っていないため評価は星4にします。
Skipping Japanese review: そんなにCPUパワーいらない素人ですが、自作歴２３年のおっさんが時々手を出したくなる、ハイエンドCPU、、、趣味の世界で楽しめます！
Skipping Japanese review: 8086Kからの乗換えなので、発熱は気にならない。10コアでの動画の書き出しは、安定して早くなった
Skipping Japanese review: 9900Kは爆熱で使いにくかった様ですが、ITXの簡易水冷で特別な設定なくMAX5.4GHz、平均5.0GHzで常用できます。ゴギガーが手軽に運用できるCPU凄過ぎ！アサシンクリードヴァルハラを1時間ほどプレイして、室温25度環境でコア温度は平均60度です。
Skipping Japanese review: 発熱はすごいが、ゲームにはもってこい
Skipping Japanese review: 最強のCPUを所有しているという優越感に浸るためのCPUです。っていうのは嘘で、当方今までi7-8700Kを使っていたのですが最近M/Bの調子も悪かったのでこれを機に新しいものに変えよう

In [44]:
df = pd.DataFrame(reviewlist)
df.to_csv(f'../data/amazon/{product_name}.csv', index=False)
print('Fin.')

Fin.
